# Introduction

On Aprit 28th, 2025, most of Spain, parts of Portugal and a tiny fraction of souther France suffered a major electrical blackout at 12:33 Madrid local time. The recovery process was slow and was performed in stages, first starting with the north and south of the Iberian peninsula and finally reaching all the country. The recovery process in the Madrid area was done in stages. While there are some parts that went back to normality at 21:30 local time, others were not recovered after 23:30

## STARS4ALL Infrastructure

The blackout affected the STARS4ALL photometers located in Spain and the server infrastructure. While the Grafana panel and its associated InfluxDB database services were not recovered until April, 29th 08:00, the core infrastructure (MQTT Broker, readings collector process and IDA database) hosted in the UCM datacenter was discoverd to be up and running entirely during the blackout. However, the progressive degradation of communications infrastructure finally affected this part too. Examination of server log records have determined that **the STARS4ALL unavailability period started at 14:00 local time (last MQTT message received) and ended at 21:40 local time (first MQTT activity recorded again)**

## Photometer selection

At the time of the blackout, most of the TESS-W traffic came from photometeres deployed all over the world. In particular, photometers deployed in Spain were sending saturated daylight readings, thus being discared before writting to the IDA database. From 12:33 to 14:00, the only photometer's readings being stored were of those recdoring night sky brightness all over the world.

To search candidates for this study, we will focus on photometers that:
1. had location deployment avaliable to us
2. were deployed in the Iberian peninsula (timezone=Europe/Madrid)
3. were sending data from 2025-04-28 21:40 (local time) on

The result list after database query is the following:

| name	| place	| town | # readings
|-------|-------|------| -----------
| stars1  | Coslada, barrio de La Estación | Coslada | 119
| stars7  | Pamplona Centro | Pamplona | 60
| stars17  | Palacio Viejo de las Corchuelas | Torrejón el Rubio, Cáceres | 45
| stars33  | M-30 | Madrid | 96
| stars62  | Centre d'Observació de l'Univers | Àger | 2
| stars73  | Observatorio Astronómico de San Blas | Teruel | 61
| stars75  | Valencia | Valencia | 58
| stars76  | Valencia | Valencia | 59
| stars85  | Observatorio Universidad Complutense de Madrid | Madrid | 95
| stars88  | Aula Espazio Gela | Bilbao | 60
| stars201  | Finca La Cocosa | Badajoz | 58
| stars202  | Casa Rural Cijara | Pantano de Cijara | 59
| stars272  | Asociación Daimieleña de Astronomía | Daimiel | 48
| stars495  | Centro Internacional de Innovación Deportiva El Anillo | Guijo de Granadilla | 56
| stars536  | Parque Natural de Izki | Korres | 60
| stars555  | Las Praderas | Las Praderas | 60
| stars608  | Centelles | Centelles | 60
| stars612  | Centre d'Observació de l'Univers | Àger | 60
| stars639  | AAS Carballada Ager | Ager | 60
| stars660  | Arzoz | Arzoz (Navarra) | 60
| stars714  | Institut d'Alcarràs | Alcarràs | 60
| stars746  | Observatorio JoseX | Sierra de Segura | 60
| stars747  | Escuela de Ingenierías Industriales de la Universidad de Extremadura | Badajoz | 50
| stars749  | Observatorio de la Vega del Codorno | Alto de la Vega | 60
| stars759  | Albanyà | Albanyà | 61
| stars795  | Astrolizard Observatory | Joanetes, Girona | 60
| stars831  | Observatori del Montsec | Sant Esteve de la Sarga | 49
| stars835  | Ayuntamiento | Oliva de la Frontera | 20
| stars945  | Observatorio Jardín | Aldeanueva de la Vera, Cáceres | 61
| stars1134  | Aniartea | Jungitu | 60



## Obtaining photometers data

Since the IDA files were being normally as usual, we will download the 2025-04 IDA monthly files for the selected photometers.

# stars1 analysis
This photometer, which belongs to Cristobal García, designer and manufacturer of [TESS-W photometers](https://tessskysensor.blogspot.com/) has been reliably sending data since 2019. It is located in Coslada, a town in the Madrid metropolitan area. As such, it can be a witness of the staged recovery. 

We begin by importing all the Python libraries we are going to use

In [10]:
import itertools
from pathlib import Path
from typing import Optional, Sequence, Iterable, Tuple

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, Column
from astropy.time import Time
import astropy.units as u
import astropy.io.ascii
from astropy.constants import astropyconst20 as const

from licatools.utils.mpl import (
    plot_single_table_column, 
    plot_single_table_columns, 
    plot_single_tables_columns,
)

Then we read the ECSV file as an Astropy table

In [7]:
STARS1_MONTHLY_FILE = Path("ecsv/stars1/stars1_2025-04.ecsv").resolve()
table = astropy.io.ascii.read(STARS1_MONTHLY_FILE, format="ecsv")

In [8]:
table

time,Enclosure Temperature,Sky Temperature,Frequency,MSAS,ZP,Sequence Number,Sun Alt,Moon Alt,Moon Illumination
,deg_C,deg_C,Hz,mag / arcsec2,,,deg,deg,
Time,float64,float64,float64,float64,float64,int64,float64,float64,float64
2025-04-01T00:00:59.000,13.94,-3.96,6.88,18.01,20.1,379483,-44.85,-20.0,0.096
2025-04-01T00:01:29.000,13.92,-4.06,6.94,18.0,20.1,379484,-44.86,-20.0,0.096
2025-04-01T00:02:00.000,13.89,-4.08,6.99,17.99,20.1,379485,-44.87,-21.0,0.096
2025-04-01T00:02:30.000,13.86,-4.13,6.97,17.99,20.1,379486,-44.88,-21.0,0.096
2025-04-01T00:03:02.000,13.82,-4.17,6.92,18.0,20.1,379487,-44.89,-21.0,0.096
2025-04-01T00:03:32.000,13.79,-4.16,7.0,17.99,20.1,379488,-44.9,-21.0,0.096
2025-04-01T00:04:03.000,13.78,-4.05,6.89,18.01,20.1,379489,-44.91,-21.0,0.096
2025-04-01T00:04:33.000,13.77,-4.16,6.93,18.0,20.1,379490,-44.91,-21.0,0.097


The first graphics will show the photometer readings on April, 28th from 21:30-to April, 29th 02:00, local time

In [44]:
low_t =  Time("2025-04-28T19:30:00", format='isot', scale='utc')
high_t = Time("2025-04-29T00:00:00", format='isot', scale='utc')
mask = np.logical_and(low_t < table["time"], table["time"] < high_t)
table = table[mask]
print(f"Selected {len(table)} entries")

Selected 119 entries


In [50]:
table.columns[0].name

AttributeError: 'Time' object has no attribute 'name'

In [48]:
plot_single_table_columns(
    table = table,
    xcolname = "time",
    ycolnames = ["MSAS", "Sequence Number"],
    title = "stars1 readings",
    legends = ["mag", "#"],
)

AttributeError: 'Time' object has no attribute 'name'